In [1]:
import os
import spacy
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
# Cargar el modelo de lenguaje en español de SpaCy
nlp = spacy.load('en_core_web_sm')

In [2]:
# Función de preprocesamiento avanzado
def advanced_preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and not token.is_stop]
    return tokens


In [3]:
# Directorio donde están los archivos
directory = "ArchivosWiki"

# Preparar los datos
sentences = []
contexts = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r", encoding='utf-8') as file:
            content = file.read()
        
        # Dividir el contenido en fragmentos más pequeños si es necesario
        max_chunk_size = 100000  # Tamaño máximo de cada fragmento
        content_chunks = [content[i:i+max_chunk_size] for i in range(0, len(content), max_chunk_size)]
        
        for chunk in content_chunks:
            # Preprocesar el fragmento
            tokens = advanced_preprocess(chunk)
            sentences.append(tokens)
            
            # Dividir el fragmento en oraciones utilizando SpaCy
            doc = nlp(chunk)
            paragraphs = [sent.text.strip() for sent in doc.sents]
            contexts.extend(paragraphs)

# Crear un DataFrame con los contextos
df = pd.DataFrame({'text': contexts})


# Generar una matriz TF-IDF para ponderar los embeddings
vectorizer = TfidfVectorizer(tokenizer=lambda x: advanced_preprocess(x), lowercase=True)
tfidf_matrix = vectorizer.fit_transform(df['text'])
tfidf_feature_names = vectorizer.get_feature_names_out()

# Crear un diccionario palabra: idf
idf_dict = dict(zip(tfidf_feature_names, vectorizer.idf_))

#Paso 1: Cargar los Embeddings de GloVe
#Primero, necesitamos cargar los embeddings desde el archivo glove.6B.300d.txt:

# Función para cargar los embeddings de GloVe
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs
            except ValueError:
                continue
    return embeddings_index

# Cargar los embeddings
glove_embeddings = load_glove_embeddings('./ModelosEmbeddings/Glove/glove.6B.300d.txt')


#Paso 2: Modificar la Función para Obtener Embeddings de Oraciones
#Actualiza la función get_weighted_sentence_embedding para utilizar los embeddings de GloVe:

def get_weighted_sentence_embedding(sentence, embeddings_index, idf_dict, vector_size=300):
    tokens = advanced_preprocess(sentence)
    word_embeddings = []
    weights = []
    for word in tokens:
        if word in embeddings_index and word in idf_dict:
            word_embeddings.append(embeddings_index[word])
            weights.append(idf_dict[word])
    if not word_embeddings:
        return np.zeros(vector_size)
    word_embeddings = np.array(word_embeddings)
    weights = np.array(weights).reshape(-1, 1)
    weighted_average = np.sum(word_embeddings * weights, axis=0) / np.sum(weights)
    return weighted_average


# Generar embeddings para cada contexto usando GloVe
df['embeddings'] = df['text'].apply(lambda x: get_weighted_sentence_embedding(x, glove_embeddings, idf_dict, vector_size=300))


def find_most_relevant_contexts(question, df, embeddings_index, idf_dict, top_n=3, vector_size=300):
    question_embedding = get_weighted_sentence_embedding(question, embeddings_index, idf_dict, vector_size)
    context_embeddings = np.vstack(df['embeddings'].values)
    similarities = cosine_similarity([question_embedding], context_embeddings)[0]
    df['similarity'] = similarities
    top_indices = df['similarity'].argsort()[-top_n:][::-1]
    most_relevant_contexts = df.iloc[top_indices]['text'].tolist()
    return most_relevant_contexts


# Ejemplo de uso
question = "What is the capital of France?"

# Encontrar los contextos más relevantes
most_relevant_contexts = find_most_relevant_contexts(question, df, glove_embeddings, idf_dict, top_n=3, vector_size=300)

# Imprimir los contextos más relevantes
print("Pregunta:", question)
print("Contextos más relevantes:")
for idx, context in enumerate(most_relevant_contexts, 1):
    print(f"{idx}. {context}")


/home/velasco/Documentos/Proyectos/MachineLearning/Tesis/SistemaQyA/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pregunta: What is the capital of France?
Contextos más relevantes:
1. "

"Paris is the capital of France.
2. Algiers became the capital of French Algeria.
3. ===National government===

As the capital of France, Paris is the seat of France's national government.


# GUARDADO DE INFORMACION

In [4]:
import pickle

# Guardar el DataFrame en un archivo pickle
with open('context_embeddings.pkl', 'wb') as f:
    pickle.dump(df, f)


In [12]:
# Convertir la columna de embeddings en una matriz numpy
embeddings_matrix = np.vstack(df['embeddings'].values)

# Guardar los embeddings en un archivo numpy
#np.save('embeddingsGlove.npy', embeddings_matrix)


In [13]:
# Guardar los textos y otros datos en un archivo CSV
#df.drop('embeddings', axis=1).to_csv('context_dataGlove.csv', index=False)


In [5]:
# Guardar idf_dict en un archivo pickle
with open('idf_dict.pkl', 'wb') as f:
    pickle.dump(idf_dict, f)


In [6]:
# Guardar los embeddings de GloVe en un archivo pickle
with open('glove_embeddings.pkl', 'wb') as f:
    pickle.dump(glove_embeddings, f)


#  LECTURA DE LOS ARCHIVOS GENERADOS

In [2]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import spacy

# Cargar el modelo de lenguaje
nlp = spacy.load('en_core_web_sm')

# Función de preprocesamiento
def advanced_preprocess(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space and not token.is_stop]
    return tokens

# Función para calcular el embedding ponderado por TF-IDF
def get_weighted_sentence_embedding(sentence, embeddings_index, idf_dict, vector_size=300):
    tokens = advanced_preprocess(sentence)
    word_embeddings = []
    weights = []
    for word in tokens:
        if word in embeddings_index and word in idf_dict:
            word_embeddings.append(embeddings_index[word])
            weights.append(idf_dict[word])
    if not word_embeddings:
        return np.zeros(vector_size)
    word_embeddings = np.array(word_embeddings)
    weights = np.array(weights).reshape(-1, 1)
    weighted_average = np.sum(word_embeddings * weights, axis=0) / np.sum(weights)
    return weighted_average

# Función para encontrar los contextos más relevantes
def find_most_relevant_contexts(question, df, embeddings_index, idf_dict, top_n=3, vector_size=300):
    question_embedding = get_weighted_sentence_embedding(question, embeddings_index, idf_dict, vector_size)
    context_embeddings = np.vstack(df['embeddings'].values)
    similarities = cosine_similarity([question_embedding], context_embeddings)[0]
    df['similarity'] = similarities
    top_indices = df['similarity'].argsort()[-top_n:][::-1]
    most_relevant_contexts = df.iloc[top_indices][['text', 'similarity']].to_dict(orient='records')
    return most_relevant_contexts

# Cargar los datos guardados
with open('ModelosEmbeddings/Glove/idf_dict.pkl', 'rb') as f:
    idf_dict = pickle.load(f)

with open('ModelosEmbeddings/Glove/glove_embeddings.pkl', 'rb') as f:
    glove_embeddings = pickle.load(f)

df = pd.read_pickle('ModelosEmbeddings/Glove/context_embeddings.pkl')

# Función para responder una pregunta
def answer_question(question, top_n=3):
    results = find_most_relevant_contexts(question, df, glove_embeddings, idf_dict, top_n=top_n, vector_size=300)
    print("\nPregunta:", question)
    print("Contextos más relevantes:")
    for idx, result in enumerate(results, 1):
        print(f"{idx}. {result['text']} (Similitud: {result['similarity']:.2f})")

# Ejemplo de uso
#question = "What is the capital of France?"
#answer_question(question, top_n=3)


In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

def answer_question_glove(
    df,
    question,
    glove_embeddings,
    idf_dict,
    top_n=3,
    gpt_model="gpt-3.5-turbo",
    max_tokens=150,
    debug=False,
    stop_sequence=None
):
    """
    Answer a question using the most relevant contexts obtained from GloVe.

    Parameters:
        df (DataFrame): The dataframe containing text and embeddings.
        question (str): The user's question.
        glove_embeddings (dict): The GloVe embeddings loaded as a dictionary.
        idf_dict (dict): Dictionary containing IDF values.
        top_n (int): Number of top relevant contexts to retrieve.
        gpt_model (str): The GPT model to use (default: "gpt-3.5-turbo").
        max_tokens (int): Maximum tokens for GPT response.
        debug (bool): If True, print debugging information.
        stop_sequence: The stop sequence for GPT response.

    Returns:
        str: The GPT-generated response based on GloVe contexts.
    """
    # Obtener los contextos más relevantes usando GloVe
    relevant_contexts = find_most_relevant_contexts(question, df, glove_embeddings, idf_dict, top_n=top_n, vector_size=300)

    # Combinar los contextos en un solo texto
    context = "\n".join([ctx['text'] for ctx in relevant_contexts])

    if debug:
        print("Contextos relevantes obtenidos de GloVe:\n")
        print(context)
        print("\n---\n")

    try:
        # Crear una consulta a GPT con los contextos obtenidos
        response = client.chat.completions.create(
            model=gpt_model,
            messages=[
                {
                    "role": "system",
                    "content": "Responde la pregunta basándote en el contexto proporcionado. Si no puedes responder basándote en el contexto, di 'Escribe AGENTE para más información.'"
                },
                {
                    "role": "system",
                    "content": f"Contexto: {context}"
                },
                {
                    "role": "user",
                    "content": f"Pregunta: {question}"
                }
            ],
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=stop_sequence,
        )
        return response.choices[0].message.content

    except Exception as e:
        print(f"Error al generar la respuesta: {e}")
        return ""



# PREGUNTAS FACTUALES

In [5]:
# Lista de preguntas
questions = [
    "Who painted the Mona Lisa?",
    "What is the capital of France?",
    "When did World War II begin?",
    "Who was the first man to walk on the Moon?",
    "What is the longest river in the world?",
    "Which country has the largest population?",
    "Who wrote 'One Hundred Years of Solitude'?",
    "What chemical element has the symbol 'O'?",
    "In what year was America discovered?",
    "What is the highest mountain in the world?",
    "Who is the author of the theory of relativity?",
    "What is the largest ocean in the world?",
    "Which country won the 2018 FIFA World Cup?",
    "Which planet is known as the Red Planet?",
    "Who was the first president of the United States?",
    "How many colors are in a rainbow?",
    "On which continent is Egypt located?",
    "What language is spoken in Brazil?",
    "What currency is used in Japan?",
    "What language is spoken in Japan?"
]


# Iterar sobre las preguntas y obtener respuestas
for i, question in enumerate(questions, 1):
    print(f"\nPregunta {i}: {question}")
    
    # Llamar al método para generar la respuesta
    response = answer_question_glove(
         df=df,
        question=question,
        glove_embeddings=glove_embeddings,
        idf_dict=idf_dict,
        top_n=20,  # Obtener los 3 contextos más relevantes
        gpt_model="gpt-3.5-turbo",
        max_tokens=150,
        debug=False  # Activa el modo de depuración para ver los contextos seleccionados
    )
    
    # Imprimir la respuesta generada
    print(f"Respuesta: {response}")



Pregunta 1: Who painted the Mona Lisa?
Respuesta: Leonardo da Vinci painted the Mona Lisa.

Pregunta 2: What is the capital of France?
Respuesta: Respuesta: The capital of France is Paris.

Pregunta 3: When did World War II begin?
Respuesta: World War II began in September 1939.

Pregunta 4: Who was the first man to walk on the Moon?
Respuesta: Duke soon descended the ladder and joined Young on the surface, becoming the tenth person to walk on the Moon.

Pregunta 5: What is the longest river in the world?
Respuesta: Respuesta: El río Nilo ha sido históricamente considerado el río más largo del mundo, aunque esta afirmación ha sido cuestionada por investigaciones que sugieren que el río Amazonas es ligeramente más largo.

Pregunta 6: Which country has the largest population?
Respuesta: Escribe AGENTE para más información.

Pregunta 7: Who wrote 'One Hundred Years of Solitude'?
Respuesta: Escribe AGENTE para más información.

Pregunta 8: What chemical element has the symbol 'O'?
Respues

# PREGUNTAS CONTEXTUALES

In [6]:
# Lista de preguntas
questions = [
    "What does 'Python' mean in programming?",
    "What does 'Python' mean in zoology?",
    "What is Java in computer science?",
    "What is Java in geography?",
    "What is the connection between the Sun and photosynthesis?",
    "How does global warming affect sea levels?",
    "What does the melting of polar ice caps imply?",
    "What is the impact of climate change on agriculture?",
    "What are the benefits of artificial intelligence in medicine?",
    "What does 'bank' mean in the financial context?",
    "What does 'bank' mean in the context of a park?",
    "How is electricity related to magnetism?",
    "What is quantum computing and how is it used?",
    "How is cryptocurrency defined?",
    "What role does water play in the life cycle?",
    "What does the 'Industrial Revolution' mean in historical terms?",
    "What is the relationship between capitalism and the free market?",
    "What does 'blockchain' mean in the technological context?",
    "What is the importance of photosynthesis in nature?",
    "What role does education play in economic development?"
]

# Iterar sobre las preguntas y obtener respuestas
for i, question in enumerate(questions, 1):
    print(f"\nPregunta {i}: {question}")
    
    # Llamar al método para generar la respuesta
    response = answer_question_glove(
         df=df,
        question=question,
        glove_embeddings=glove_embeddings,
        idf_dict=idf_dict,
        top_n=20,  # Obtener los 3 contextos más relevantes
        gpt_model="gpt-3.5-turbo",
        max_tokens=150,
        debug=False  # Activa el modo de depuración para ver los contextos seleccionados
    )
    
    # Imprimir la respuesta generada
    print(f"Respuesta: {response}")




Pregunta 1: What does 'Python' mean in programming?
Respuesta: En programación, 'Python' se refiere a un lenguaje de programación de alto nivel y propósito general.

Pregunta 2: What does 'Python' mean in zoology?
Respuesta: Escribe AGENTE para más información.

Pregunta 3: What is Java in computer science?
Respuesta: En computer science, Java es un lenguaje de programación que puede ser compilado en bytecode de Java y ejecutado en una Java Virtual Machine (JVM). También es parte de la plataforma Java, que incluye la JVM y varias APIs de Java.

Pregunta 4: What is Java in geography?
Respuesta: Java in geography refers to an island of Indonesia.

Pregunta 5: What is the connection between the Sun and photosynthesis?
Respuesta: La conexión entre el Sol y la fotosíntesis es que la energía del Sol es capturada por la fotosíntesis y utilizada por los organismos vivos para llevar a cabo este proceso. La fotosíntesis permite que la energía solar sea recolectada directamente por las formas de

# ANALOGIAS

In [7]:
from scipy.spatial.distance import cosine

# Función para resolver analogías con embeddings GloVe
def find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=1):
    """
    Encuentra la palabra que completa la analogía: word_a es a word_b como word_c es a ?

    Parámetros:
        word_a (str): Primera palabra en la analogía (e.g., "rey").
        word_b (str): Segunda palabra en la analogía (e.g., "reina").
        word_c (str): Tercera palabra en la analogía (e.g., "hombre").
        glove_embeddings (dict): Diccionario de embeddings de GloVe.
        top_n (int): Número de palabras más cercanas a retornar.

    Retorna:
        list: Lista de palabras más cercanas junto con sus similitudes.
    """
    try:
        # Convertir las palabras a minúsculas
        word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()

        # Verificar si las palabras están en el vocabulario
        for word in [word_a, word_b, word_c]:
            if word not in glove_embeddings:
                return [f"'{word}' no está en el vocabulario del modelo."]

        # Calcular el vector resultante de la analogía
        analogy_vector = glove_embeddings[word_a] - glove_embeddings[word_b] + glove_embeddings[word_c]

        # Encontrar las palabras más similares al vector resultante
        similarities = {
            word: 1 - cosine(embedding, analogy_vector)
            for word, embedding in glove_embeddings.items()
            if word not in {word_a, word_b, word_c}
        }

        # Ordenar por similitud y seleccionar las mejores
        sorted_words = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
        return sorted_words[:top_n]

    except Exception as e:
        return [f"Error al procesar la analogía: {e}"]



In [8]:
# PREGUNTAS DE ANALOGÍAS
word_a = "king"
word_b = "queen"
word_c = "man"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")


'king' es a 'queen' como 'man' es a:
- himself (similitud: 0.5258)
- brother (similitud: 0.5021)
- father (similitud: 0.5020)


In [9]:
# PREGUNTAS DE ANALOGÍAS
word_a = "madrid"
word_b = "spain"
word_c = "tokio"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")


'madrid' es a 'spain' como 'tokio' es a:
- mitsui (similitud: 0.3884)
- yasuda (similitud: 0.3713)
- wilsher (similitud: 0.3638)


In [10]:
# PREGUNTAS DE ANALOGÍAS
word_a = "day"
word_b = "sunny"
word_c = "night"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")

'day' es a 'sunny' como 'night' es a:
- week (similitud: 0.6522)
- friday (similitud: 0.6124)
- after (similitud: 0.6091)


In [11]:
# PREGUNTAS DE ANALOGÍAS
word_a = "spoon"
word_b = "soup"
word_c = "fork"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")

'spoon' es a 'soup' como 'fork' es a:
- spatula (similitud: 0.5002)
- forks (similitud: 0.4579)
- tines (similitud: 0.4309)


In [12]:
# PREGUNTAS DE ANALOGÍAS
word_a = "apple"
word_b = "fruit"
word_c = "dog"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")

'apple' es a 'fruit' como 'dog' es a:
- cat (similitud: 0.4752)
- dogs (similitud: 0.4635)
- iphone (similitud: 0.4492)


In [13]:
# PREGUNTAS DE ANALOGÍAS
word_a = "eye"
word_b = "see"
word_c = "ear"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")

'eye' es a 'see' como 'ear' es a:
- ears (similitud: 0.5975)
- nose (similitud: 0.5061)
- throat (similitud: 0.4963)


In [14]:
# PREGUNTAS DE ANALOGÍAS
word_a = "car"
word_b = "fuel"
word_c = "human"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")

'car' es a 'fuel' como 'human' es a:
- rights (similitud: 0.4648)
- woman (similitud: 0.4481)
- man (similitud: 0.4365)


In [15]:
# PREGUNTAS DE ANALOGÍAS
word_a = "bird"
word_b = "flying"
word_c = "fish"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")

'bird' es a 'flying' como 'fish' es a:
- poultry (similitud: 0.5474)
- birds (similitud: 0.5456)
- seafood (similitud: 0.5294)


In [16]:
# PREGUNTAS DE ANALOGÍAS
word_a = "computer"
word_b = "processor"
word_c = "human"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")

'computer' es a 'processor' como 'human' es a:
- experts (similitud: 0.5135)
- scientists (similitud: 0.4921)
- rights (similitud: 0.4825)


In [17]:
# PREGUNTAS DE ANALOGÍAS
word_a = "book"
word_b = "read"
word_c = "movie"

result = find_analogy_glove(word_a, word_b, word_c, glove_embeddings, top_n=3)

# Mostrar resultados
print(f"'{word_a}' es a '{word_b}' como '{word_c}' es a:")
for word, similarity in result:
    print(f"- {word} (similitud: {similarity:.4f})")

'book' es a 'read' como 'movie' es a:
- film (similitud: 0.7491)
- sequel (similitud: 0.6579)
- films (similitud: 0.6530)
